# 下载公告

地理标志产品检索网站：https://ggfw.cnipa.gov.cn/dlbzsq/dbQuery

需要从网站中爬取公告作为初步数据。

因为网站源代码中没有``URL``可供爬取，只能通过点击跳转到相应公告，所以想法是使用``selenium``来模拟浏览器点击，自动的下载文件。

In [ ]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

def window_switch_to_new(driver, original_window):
    """跳转到新窗口"""
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            return
        
def window_switch_to_original(driver, original_window):
    """跳转到原始窗口"""
    driver.close()
    driver.switch_to.window(original_window)

def log_write(file, content):
    """写入日志文件"""
    with open(file, "a") as f:
        f.write(content + "\n")

# 日志文件, 记录已经下载的pdf文件
log = "data/log.txt"
saved = [f.strip() for f in open(log, "r").readlines()]

# 浏览器配置
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.prompt_for_download": False, # 禁用下载提示框
    "plugins.always_open_pdf_externally": True # 禁用 PDF Viewer 插件
}
chrome_options.add_experimental_option("prefs", prefs)

# 启动浏览器
driver = webdriver.Chrome(options=chrome_options)

进入检索目录

In [ ]:
# 访问网页
driver.get("https://ggfw.cnipa.gov.cn/dlbzsq/dbQuery")

# 跳转到文件目录页面
# button = driver.find_element(By.XPATH, "/html/body/div/div/div[2]/div/div[2]/div/div[1]/button") # 新目录
button = driver.find_element(By.XPATH, "/html/body/div/div/div[2]/div/div[2]/div/div[2]/button") # 旧目录

button.click()
WebDriverWait(driver, 20)

# 记录原始窗口
original_window = driver.current_window_handle

爬取当前页面的所有文件

缺点：无法定位新页面，需要手动进入新页面

In [ ]:
# 获取所有文件页面的链接
links = driver.find_elements(By.CLASS_NAME, "policyAnswersItem")

# 依次处理每一个文件
for link in links:
    # 跳转到新页面
    link.click()
    WebDriverWait(driver, 10)
    window_switch_to_new(driver, original_window)

    # PDF文件
    pdf_button = driver.find_element(By.CLASS_NAME, "el-link--inner")
    file_name = pdf_button.text
    
    # 判断是否要保存
    if file_name in saved:
        pass
    else:
        pdf_button.click()
        sleep(3)
        log_write(log, file_name)
        saved.append(file_name)

    # 跳转回起始页面
    window_switch_to_original(driver, original_window)
    sleep(1.5)